In [206]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

In [207]:
data_games = pd.read_parquet("dataSet/steam_game_clean.parquet")

In [208]:
data_games.head(5)

,publisher,genres,app_name,title,price,early_access,id,developer,Year
0,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,4.99,False,761140,Kotoshiro,2018.0
1,Kotoshiro,Casual,Lost Summoner Kitty,Lost Summoner Kitty,4.99,False,761140,Kotoshiro,2018.0
2,Kotoshiro,Indie,Lost Summoner Kitty,Lost Summoner Kitty,4.99,False,761140,Kotoshiro,2018.0
3,Kotoshiro,Simulation,Lost Summoner Kitty,Lost Summoner Kitty,4.99,False,761140,Kotoshiro,2018.0
4,Kotoshiro,Strategy,Lost Summoner Kitty,Lost Summoner Kitty,4.99,False,761140,Kotoshiro,2018.0


In [209]:
#toma una columna 'title', para filtrar todos los duplicados luego tomamos el primer valor que nos da.
data_games.drop_duplicates(subset='title', keep='first', inplace=True)

In [210]:
data_games

,publisher,genres,app_name,title,price,early_access,id,developer,Year
0,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,4.99,False,761140,Kotoshiro,2018.0
5,"Making Fun, Inc.",Free to Play,Ironbound,Ironbound,0.00,False,643980,Secret Level SRL,2018.0
9,Poolians.com,Casual,Real Pool 3D - Poolians,Real Pool 3D - Poolians,0.00,False,670290,Poolians.com,2017.0
14,彼岸领域,Action,弹炸人2222,弹炸人2222,0.99,False,767400,彼岸领域,2017.0
17,Trickjump Games Ltd,Action,Battle Royale Trainer,Battle Royale Trainer,3.99,False,772540,Trickjump Games Ltd,2018.0
...,...,...,...,...,...,...,...,...,...
71485,Bidoniera Games,Action,Kebab it Up!,Kebab it Up!,1.99,False,745400,Bidoniera Games,2018.0
71489,Ghost_RUS Games,Casual,Colony On Mars,Colony On Mars,1.99,False,773640,"Nikita ""Ghost_RUS""",2018.0
71493,Sacada,Casual,LOGistICAL: South Africa,LOGistICAL: South Africa,4.99,False,733530,Sacada,2018.0
71496,Laush Studio,Indie,Russian Roads,Russian Roads,1.99,False,610660,Laush Dmitriy Sergeevich,2018.0


In [211]:
data_games = data_games.sample(n=1000, random_state=40)

In [212]:
data_games.head()

,publisher,genres,app_name,title,price,early_access,id,developer,Year
53952,Available,Indie,Fantasy Grounds - Top-Down Tokens - Darkwoulfe...,Fantasy Grounds - Top-Down Tokens - Darkwoulfe...,4.99,False,450096,"SmiteWorks USA, LLC",2016.0
27611,"Rocketeer Games Studio, LLC",Action,Red Crucible®: Reloaded,Red Crucible®: Reloaded,0.00,False,732730,"Rocketeer Games Studio, LLC",2015.0
48408,Red Beat,Indie,Space Rogue,Space Rogue,9.99,False,364300,Red Beat,2016.0
441,"LucasArts, Lucasfilm, Disney Interactive",Adventure,Indiana Jones® and the Fate of Atlantis™,Indiana Jones® and the Fate of Atlantis™,5.99,False,6010,LucasArts,1992.0
32621,"Strata Mixed Reality, Inc.",Animation &amp; Modeling,Strata Spaces VR – Professional Edition 7-Day ...,Strata Spaces VR – Professional Edition 7-Day ...,0.00,False,719010,"Strata Mixed Reality, Inc.",2017.0


In [213]:
data_games.shape

(1000, 9)

In [214]:
data_games.dtypes

publisher        object
genres           object
app_name         object
title            object
price           float64
early_access       bool
id               object
developer        object
Year            float64
dtype: object

In [215]:
# Primero, vamos a convertir la columna 'nombre' a una representación numérica usando TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(data_games['genres'])


In [216]:
# Ahora, calculamos la matriz de similitud de coseno
similarity_matrix = cosine_similarity(tfidf_matrix)



In [217]:
data_games['id'].head(25)

53952    450096
27611    732730
48408    364300
441        6010
32621    719010
20383    592660
18281    555320
29999    726650
6721     342804
10008    414673
28937    740800
69321    215242
63468    329821
2384     213892
7378     364800
66614    271444
39712    607400
9226     274350
10169    411760
54312    441781
30647    688890
66134    275806
19987    610222
20280    578870
10448    390396
Name: id, dtype: object

In [238]:
# Para hacer recomendaciones, puedes buscar los productos más similares a un producto dado
genres = 'Action'  # El nombre del producto para el que quieres hacer recomendaciones
producto = data_games[data_games['genres'] == genres]
item_id = '6010'


if not producto.empty:
    product_index = producto.index[0]
    genres_p = producto['genres']
    #instanceo el vectoridador para utilizarlo
    vector = TfidfVectorizer()
    #tranforma en numeros las palabras para que el modelo entienda 
    matriz = vector.fit_transform(genres_p)
    product_similarities = cosine_similarity(matriz)
    product_similarities = list(enumerate(product_similarities))
    similar_games = [producto.iloc[i[0]] for i in product_similarities]

    #print("Los productos más similares al producto son:")
    #print (similar_games[1:6])
else:
    print("Producto no encontrado")
games = [i['title'] for i in similar_games]
dictionary = {'los 5 juego mas similares': games[:5]}
print(dictionary)


{'los 5 juego mas similares': ['Red Crucible®: Reloaded', 'Sleeping Dogs: Top Dog Gold Pack', 'Star Trek™: Judgment Rites', 'Necrosphere', 'Devilian: Champion Pack']}


In [236]:
games = [i['title'] for i in similar_games]
games[:5]

['Red Crucible®: Reloaded',
 'Sleeping Dogs: Top Dog Gold Pack',
 'Star Trek™: Judgment Rites',
 'Necrosphere',
 'Devilian: Champion Pack']

In [86]:
data_item = pd.read_parquet("dataSet/user_items_clean.parquet")

In [88]:
data_item.head(5)

,item_id,item_name,playtime_forever,steam_id,items_count,user_id,user_url
0,10,Counter-Strike,6,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,Deathmatch Classic,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
4,50,Half-Life: Opposing Force,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...


In [89]:
# Para hacer recomendaciones, puedes buscar los productos más similares a un producto dado
id_user = 'diet_cola'  # El nombre del producto para el que quieres hacer recomendaciones
producto = data_item[data_item['user_id'] == id_user]  # Asegúrate de que 'user_id' sea el nombre correcto de la columna
producto

,item_id,item_name,playtime_forever,steam_id,items_count,user_id,user_url
2372892,220,Half-Life 2,298,76561198024832751,269,diet_cola,http://steamcommunity.com/id/diet_cola
2372893,340,Half-Life 2: Lost Coast,1,76561198024832751,269,diet_cola,http://steamcommunity.com/id/diet_cola
2372894,240,Counter-Strike: Source,103,76561198024832751,269,diet_cola,http://steamcommunity.com/id/diet_cola
2372895,4000,Garry's Mod,64613,76561198024832751,269,diet_cola,http://steamcommunity.com/id/diet_cola
2372896,6220,FlatOut,96,76561198024832751,269,diet_cola,http://steamcommunity.com/id/diet_cola
...,...,...,...,...,...,...,...
2373156,205950,Jet Set Radio,0,76561198024832751,269,diet_cola,http://steamcommunity.com/id/diet_cola
2373157,275850,No Man's Sky,275,76561198024832751,269,diet_cola,http://steamcommunity.com/id/diet_cola
2373158,462930,AdVenture Communist,6,76561198024832751,269,diet_cola,http://steamcommunity.com/id/diet_cola
2373159,394360,Hearts of Iron IV,2878,76561198024832751,269,diet_cola,http://steamcommunity.com/id/diet_cola


In [114]:
# Para hacer recomendaciones, puedes buscar los productos más similares a un producto dado
id_user = 'diet_cola'  # El nombre del producto para el que quieres hacer recomendaciones
producto = data_item[data_item['user_id'] == id_user]  # Asegúrate de que 'user_id' sea el nombre correcto de la columna

#Buscar los géneros de esos juegos en la base de datos steam_games
juegos_con_generos = data_games[data_games['id'].isin(producto['item_id'])]

# Extraer solo las columnas que necesitas, por ejemplo 'item_id' y 'genres'
juegos_generos = juegos_con_generos[['id', 'genres']]

if not producto.empty:
    product_index = producto.index[0]
    genres = juegos_generos['genres']
    vector = TfidfVectorizer()
    matriz = vector.fit_transform(genres)
    product_similarities = cosine_similarity(matriz)
    product_similarities = list(enumerate(product_similarities))
    similar_games = [producto.iloc[i[0]] for i in product_similarities]
    print("Los productos más similares al producto son:")
    print (similar_games[1:6])
else:
    print("Producto no encontrado")


Los productos más similares al producto son:
[item_id                                                340
item_name                          Half-Life 2: Lost Coast
playtime_forever                                         1
steam_id                                 76561198024832751
items_count                                            269
user_id                                          diet_cola
user_url            http://steamcommunity.com/id/diet_cola
Name: 2372893, dtype: object, item_id                                                240
item_name                           Counter-Strike: Source
playtime_forever                                       103
steam_id                                 76561198024832751
items_count                                            269
user_id                                          diet_cola
user_url            http://steamcommunity.com/id/diet_cola
Name: 2372894, dtype: object, item_id                                               4000
item_name

In [115]:
modelo = './dataSet/base_modelo_clean.parquet'
data_review.to_parquet(modelo, index=False)
print(f'Se guardó el archivo {modelo}')

Se guardó el archivo ./dataSet/base_modelo_clean.parquet


In [153]:
def recomendacion_juego(id_de_producto:str):
    modelo = pd.read_parquet('dataSet/steam_game_clean.parquet')
    
    game = modelo[modelo['id'] == id_de_producto]
    # Extraer el género del juego
    if game.empty:
        print("Producto no encontrado")
        return
    
    genres = game['genres'].iloc[0]  # Obtener el primer elemento, ya que 'genres' es una serie
    
    # Para hacer recomendaciones, puedes buscar los productos más similares a un producto dado
    
    producto = modelo[modelo['genres'] == genres]

    if not producto.empty:
        product_index = producto.index[0]
        genres_p = producto['genres']
        #instanceo el vectoridador para utilizarlo
        vector = TfidfVectorizer()
        #tranforma en numeros las palabras para que el modelo entienda
        matriz = vector.fit_transform(genres_p)
        product_similarities = cosine_similarity(matriz)
        product_similarities = list(enumerate(product_similarities))
        similar_games = [producto.iloc[i[0]] for i in product_similarities]
        print("Los productos más similares al producto son:")
        print (similar_games[1:6])
    else:
        print("Producto no encontrado")
        
    return similar_games


In [154]:
recomendacion_juego('6010')

Los productos más similares al producto son:
[publisher         Trickjump Games Ltd
genres                      Adventure
app_name        Battle Royale Trainer
title           Battle Royale Trainer
price                            3.99
early_access                    False
id                             772540
developer         Trickjump Games Ltd
Year                           2018.0
Name: 18, dtype: object, publisher                          Stegalosaurus Game Development
genres                                                  Adventure
app_name        Army of Tentacles: (Not) A Cthulhu Dating Sim:...
title           Army of Tentacles: (Not) A Cthulhu Dating Sim:...
price                                                       10.99
early_access                                                False
id                                                         770380
developer                          Stegalosaurus Game Development
Year                                                       